In [ ]:
import os
from os import path
import numpy as np
import pandas as pd
import math
from IPython.display import clear_output
from datetime import datetime
import matplotlib
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
import scipy
from scipy.signal import savgol_filter
# Import curve fitting package from scipy
from scipy.optimize import curve_fit

import os
import math
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize, signal
from lmfit import models


In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
#This script can fit XRD for perovskites from 11-21 degrees and plot the data. Scan parameters: 0.02 spacing, 1 degree/min, 11-21 degrees

#Does not include fitting for 13.2 degree peak seen when films are severely over-reacted due to MAI ITO reaction

#Setting parameters for plots
n=5
params = {'legend.fontsize': 20,
         'axes.labelsize': 30,
         'axes.titlesize':'x-large',
         'xtick.labelsize':'40',
         'ytick.labelsize':'40',
         'font.weight':'bold',
         'figure.figsize' : (16,12)}
plt.rcParams.update(params)

#Change directory to the location of the data file
directory = "D:/AKXRD/2021-Perovskites-Cleaned/CS5XX/PIN/"
os.chdir(directory)

#Select ETL
etl='C60'

#Create Empty Arrays For files
XRDFiles=[]
stuff=[]

for filename in os.listdir(directory):
        if filename.endswith('.TXT'):
            df=pd.read_csv(filename,sep='\t',skiprows=17, names=['Degrees','Intensity'],encoding='cp1252')
            newfilename=filename[:-4]
            df['Filename']=newfilename
            print(newfilename)
            XRDFiles.append(df)

#Specify file, read in file tab delimited, skip first 17 rows of header info, no header
for df in XRDFiles:
    datak=df
    x= datak["Degrees"]
    y=datak["Intensity"]
    filename=datak['Filename'][0]

    #subtract baseline
    ymin=np.min(y)
    baseline=np.median(y)
    #baseline=16
    print('the minimum value is ',ymin)
    ynew=[]
    for i in range(len(y)):
        ynew.append(y[i]-baseline)
    for i in range(len(ynew)):
        if ynew[i]<0:
            ynew[i]=1

    #apply a savitsky golay filter to the data (smooth data) for plotting only
    yhat = moving_average(ynew, n)
    xhat=x[n-1:]

    #Plot XRD data on log scale
    asize=1
    linsize=2
    plt.plot(xhat,yhat,'k',)
    plt.plot([12.1,12.1],[0,1e5],color='k', markersize=asize,linewidth=linsize, linestyle='dotted')
    plt.plot([13.1,13.1],[0,1e5],color='k', markersize=asize,linewidth=linsize, linestyle='dotted')
    plt.plot([13.5,13.5],[0,1e5],color='k', markersize=asize,linewidth=linsize, linestyle='dotted')
    plt.plot([14.5,14.5],[0,1e5],color='k', markersize=asize,linewidth=linsize, linestyle='dotted')
    plt.plot([19.5,19.5],[0,1e5],color='k', markersize=asize,linewidth=linsize, linestyle='dotted')
    plt.plot([20.5,20.5],[0,1e5],color='k', markersize=asize,linewidth=linsize, linestyle='dotted')

    plt.yscale('log')
    #plt.ylim(0,.1)
    #plt.xlim(11,21)
    plt.ylim(1,100000)
    plt.title("$Log$ $Scale$ $XRD$", fontsize=40)
    plt.xlabel('$Angle$ $(Degrees)$', fontsize = 40)
    plt.ylabel('$Intensity$ $(arb.)$', fontsize = 40)
    #plt.xticks([12,13,13.5,14.5,19.5,20.5])
    print(filename[0])
    plt.savefig(filename+'logxrd.png')
    #plt.savefig(filename+'logxrd.png')
    plt.show()

    #These are scaling factors for films
    #zscale is from a powder pattern to adjust for structure factor of peak 1 and 2
    #pbscale is taken from a precursor film on each substrate
    #MAIscale is taken from a ~ fully reacted film on each substrate
    zscale=9.7
    if etl=='CdS':
        pbscale=629
        maiscale=283
    elif etl=='SnO2':
        pbscale=14823
        maiscale=725
    elif etl=='C60':
        pbscale=6751
        maiscale=686
    elif etl=='SLG':
        pbscale=50180
        maiscale=2949.5

    #numerical integration and calculation of parameters
    pb=np.trapz(y=yhat[49:99],x=xhat[44:94])
    mapi1=np.trapz(y=yhat[119:169],x=xhat[119:169])
    mapi2=np.trapz(y=yhat[419:469],x=xhat[419:469])
    #oldy=pb/(mapi1+mapi2*zscale)
    x=(pb)/(pbscale)
    y=(pb/pbscale)/(pb/pbscale+(mapi1+mapi2*zscale)/maiscale)
    z=mapi1/mapi2


    templist=[filename,pb,mapi1,mapi2,x,y,z]
    
    
    stuff.append(templist)
    stuff.append(templist)
    stuff.append(templist)
    stuff.append(templist)    

newstuff=pd.DataFrame(stuff,columns=['Filename','Pbint','MAPI1int','MAPI2int','Y','Z'])
newstuff.to_csv(filename+'xrdint.csv')
print(stuff)


#for diagnosis to choose scan ranges
#print(xhat[44:94])
#print(xhat[119:169])
#print(xhat[419:469])